# Reflect Tables into SQLAlchemy ORM

In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


In [2]:
# create engine to ev_db.sqlite
engine = create_engine("sqlite:///oil_price_interest_db.sqlite")


In [3]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)
# View all of the classes that automap found
Base.classes.keys()

['CadRates', 'FedRates', 'crude_oil_prices']

In [4]:
# Save references to each table using correct case
crude_oil_prices = Base.classes.crude_oil_prices  
cad_rates = Base.classes.CadRates
fed_rates = Base.classes.FedRates

In [5]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Data loading 

In [14]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LinearRegression

In [ ]:
min_year_WTI = session.query(func.min(crude_oil_prices.Year)).filter(crude_oil_prices.WTI.isnot(None)).scalar()
max_year_WTI = session.query(func.max(crude_oil_prices.Year)).filter(crude_oil_prices.WTI.isnot(None)).scalar()
min_year_WCS = session.query(func.min(crude_oil_prices.Year)).filter(crude_oil_prices.WCS.isnot(None)).scalar()
max_year_WCS = session.query(func.max(crude_oil_prices.Year)).filter(crude_oil_prices.WCS.isnot(None)).scalar()
min_year_fed = session.query(func.min(fed_rates.Year)).scalar()
min_year_cad = session.query(func.min(cad_rates.Year)).scalar()
print(f"Minimum Year (WTI not null): {min_year}")
print(f"Maximum Year (WTI not null): {max_year}")
print(f"Minimum Year (WCS not null): {min_year}")
print(f"Maximum Year (WCS not null): {max_year}")
print (f'min year in cad dataset :{min_year_cad}, minimun year in fed dataset: {min_year_fed}')

Minimum Year (WTI not null): 2005
Maximum Year (WTI not null): 2025
Minimum Year (WCS not null): 2005
Maximum Year (WCS not null): 2025
min year in cad dataset :2014, minimun year in fed dataset 2012


In [ ]:
# Query for crude oil prices WCS
crude_oil_prices_cad = session.query(
    crude_oil_prices.Year,
    crude_oil_prices.Month,
    crude_oil_prices.WCS,
).filter(
    crude_oil_prices.Year >= 2014,
    crude_oil_prices.Year <= 2024
).all()

# Convert the crude oil prices query result to a DataFrame
df_1 = pd.DataFrame(crude_oil_prices_cad, columns=['Year', 'Month', 'WCS'])

In [24]:
# Query for CAD rates
cad_rate = session.query(
    cad_rates.Year,
    cad_rates.Month,
    cad_rates.Inflation,
    cad_rates.Ten_Year_Bond_Yield,
    cad_rates.Overnight_Rate
).filter(
    cad_rates.Year >= 2014,
    cad_rates.Year <= 2024
).all()

# Convert the CAD rates query result to a DataFrame
df_2 = pd.DataFrame(cad_rate, columns=['Year', 'Month', 'Inflation', 'Ten_Year_Bond_Yield', 'Overnight Rate'])


In [25]:
#merge df_1 and df_2 based on Year Month
crude_oil_cad = pd.merge(df_1, df_2, on=['Year', 'Month'], how='inner')


In [30]:
# Query for crude oil prices WTI
crude_oil_prices_cad = session.query(
    crude_oil_prices.Year,
    crude_oil_prices.Month,
    crude_oil_prices.WTI,
).filter(
    crude_oil_prices.Year >= 2014,
    crude_oil_prices.Year <= 2024
).all()

# Convert the crude oil prices query result to a DataFrame
df_3 = pd.DataFrame(crude_oil_prices_cad, columns=['Year', 'Month', 'WTI'])

In [31]:
# Query for Fed rates
fed_rate = session.query(
    fed_rates.Year,
    fed_rates.Month,
    fed_rates.Inflation,
    fed_rates.Ten_Year_Bond_Yield,
    fed_rates.Federal_Rate
).filter(
    fed_rates.Year >= 2014,
    fed_rates.Year <= 2024
).all()

# Convert the CAD rates query result to a DataFrame
df_4 = pd.DataFrame(fed_rate, columns=['Year', 'Month', 'Inflation', 'Ten_Year_Bond_Yield', 'Federal Rate'])


In [32]:
#merge df_3 and df_4 on Year and month
crude_oil_fed = pd.merge(df_3, df_4, on=['Year', 'Month'], how='inner')


In [56]:
crude_oil_cad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Year                 264 non-null    int64 
 1   Month                264 non-null    int64 
 2   WCS                  264 non-null    object
 3   Inflation            264 non-null    object
 4   Ten_Year_Bond_Yield  264 non-null    object
 5   Overnight Rate       264 non-null    object
dtypes: int64(2), object(4)
memory usage: 12.5+ KB


# Data Preparation

In [37]:
#seperate X and target y
y =crude_oil_fed['WTI']
X = crude_oil_fed.drop(columns=['WTI', 'Year', 'Month'])

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(99, 3)

# Build a Linear Regression Model

In [41]:
#create a model with scikit-learn
model=LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [42]:
print(f"Model's slope: {model.coef_}")
print(f"Model's y-intercept: {model.intercept_}")

Model's slope: [38.80916961  9.29493634 -3.73986082]
Model's y-intercept: -29.45202662604933


In [50]:
predicted_y_values=model.predict(X_test)


# Evaluate the Model

In [55]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, predicted_y_values)
r2 = r2_score(y_test, predicted_y_values)
rmse= np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')
print(f"The root mean squared error is {rmse}.")


Mean Squared Error: 98.57328700515399
R² Score: 0.6894392881816354
The root mean squared error is 9.928408080108008.
